# Face recognition library

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import os
import uuid
import cv2
import numpy as np
from PIL import Image, ImageTk
import face_recognition
import qrcode
import json

# =========================
# CONFIG
# =========================
FACE_DB = {}  # sid -> { name, uuid, encoding_path, qr }
DATA_FILE = "face_db.json"
ENCODINGS_DIR = "encodings"
QR_DIR = "qr_codes"

os.makedirs(ENCODINGS_DIR, exist_ok=True)
os.makedirs(QR_DIR, exist_ok=True)


# =========================
# DATA PERSISTENCE
# =========================
def save_index():
    """Save FACE_DB to disk as JSON."""
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(FACE_DB, f, indent=2)


def load_index():
    """Load FACE_DB from disk if file exists."""
    global FACE_DB
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r", encoding="utf-8") as f:
            FACE_DB = json.load(f)
    else:
        FACE_DB = {}


def enc_path_for(sid):
    return os.path.join(ENCODINGS_DIR, f"{sid}.npy")


def ensure_single_face(locs):
    return len(locs) == 1


def load_by_uuid(uid):
    """Return (sid, info) by uuid or (None, None) if not found."""
    for sid, info in FACE_DB.items():
        if info.get("uuid") == uid:
            return sid, info
    return None, None


# =========================
# CAMERA VIEW BASE
# =========================
class CameraView(tk.Toplevel):
    def _init_(self, master, title="Camera"):
        super()._init_(master)
        self.title(title)
        self.protocol("WM_DELETE_WINDOW", self._on_close)
        self.frame = None
        self.callback = None

        self.video_label = ttk.Label(self)
        self.video_label.pack()

        self.status_var = tk.StringVar()
        self.status_label = ttk.Label(self, textvariable=self.status_var)
        self.status_label.pack()

        self.buttons = ttk.Frame(self)
        self.buttons.pack(pady=6)

        self.cap = cv2.VideoCapture(0)
        self._update_frame()

    def set_status(self, text):
        self.status_var.set(text)

    def set_callback(self, func):
        self.callback = func

    def _update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            self.frame = frame.copy()
            display_frame = frame
            if self.callback:
                display_frame = self.callback(frame.copy())
            rgb = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
        self.after(30, self._update_frame)

    def _on_close(self):
        if hasattr(self, "cap") and self.cap.isOpened():
            self.cap.release()
        self.destroy()


# =========================
# REGISTER FACE (PHOTO or SCAN)
# =========================
class RegisterFaceDialog(tk.Toplevel):
    def _init_(self, master):
        super()._init_(master)
        self.title("Register Face")
        self.resizable(False, False)
        self.protocol("WM_DELETE_WINDOW", self.destroy)

        frm = ttk.Frame(self, padding=12)
        frm.pack(fill="both", expand=True)

        ttk.Label(frm, text="Student ID:").grid(row=0, column=0, sticky="e", padx=(0, 8), pady=6)
        ttk.Label(frm, text="Name:").grid(row=1, column=0, sticky="e", padx=(0, 8), pady=6)

        self.sid_var = tk.StringVar()
        self.name_var = tk.StringVar()

        ttk.Entry(frm, textvariable=self.sid_var, width=28).grid(row=0, column=1, pady=6)
        ttk.Entry(frm, textvariable=self.name_var, width=28).grid(row=1, column=1, pady=6)

        self.photo_label = ttk.Label(frm)
        self.photo_label.grid(row=3, column=0, columnspan=2, pady=(10, 0))

        btns = ttk.Frame(frm)
        btns.grid(row=2, column=0, columnspan=2, pady=(10, 0))
        ttk.Button(btns, text="Use Photo", command=self._use_photo).grid(row=0, column=0, padx=6)
        ttk.Button(btns, text="Scan with Camera", command=self._scan_camera).grid(row=0, column=1, padx=6)
        ttk.Button(btns, text="Close", command=self.destroy).grid(row=0, column=2, padx=6)

    def _validate_inputs(self):
        sid = self.sid_var.get().strip()
        name = self.name_var.get().strip()
        if not sid or not name:
            messagebox.showerror("Error", "Student ID and Name are required.")
            return None, None
        return sid, name

    def _use_photo(self):
        sid, name = self._validate_inputs()
        if not sid:
            return

        file_path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        if not file_path:
            return

        try:
            rgb = face_recognition.load_image_file(file_path)
            locs = face_recognition.face_locations(rgb)

            if not ensure_single_face(locs):
                messagebox.showerror("Error", "Need exactly one face in the photo.")
                return

            encs = face_recognition.face_encodings(rgb, locs)
            if not encs:
                messagebox.showerror("Error", "Face detected but encoding failed.")
                return

            uid = FACE_DB.get(sid, {}).get("uuid") or str(uuid.uuid4())

            np.save(enc_path_for(sid), encs[0])

            FACE_DB[sid] = {
                "name": name,
                "uuid": uid,
                "encoding_path": enc_path_for(sid),
                **({"qr": FACE_DB[sid].get("qr")} if sid in FACE_DB else {})
            }
            save_index()

            img = Image.open(file_path).resize((150, 150))
            photo = ImageTk.PhotoImage(img)
            self.photo_label.config(image=photo)
            self.photo_label.image = photo

            messagebox.showinfo("Success", "Photo uploaded and face data saved!")

        except Exception as e:
            messagebox.showerror("Error", f"Failed to process the photo.\n{str(e)}")

    def _scan_camera(self):
        sid, name = self._validate_inputs()
        if not sid:
            return

        cam = CameraView(self, title="Register Face - Camera")
        cam.set_status("Ensure only ONE face is visible. Click 'Save Face' when ready.")

        def on_frame(frame_bgr):
            rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            locs = face_recognition.face_locations(rgb)
            for (top, right, bottom, left) in locs:
                cv2.rectangle(frame_bgr, (left, top), (right, bottom), (0, 255, 0), 2)
            if len(locs) == 0:
                cam.set_status("No face detected...")
            elif len(locs) > 1:
                cam.set_status("Multiple faces detected! Only one person in view.")
            else:
                cam.set_status("One face detected. Click 'Save Face'.")
            return frame_bgr

        cam.set_callback(on_frame)

        def save_face():
            if cam.frame is None:
                return
            rgb = cv2.cvtColor(cam.frame, cv2.COLOR_BGR2RGB)
            locs = face_recognition.face_locations(rgb)
            if not ensure_single_face(locs):
                messagebox.showerror("Error", "Need exactly one face to save.")
                return
            encs = face_recognition.face_encodings(rgb, locs)
            if not encs:
                messagebox.showerror("Error", "Face detected but encoding failed.")
                return

            uid = FACE_DB.get(sid, {}).get("uuid") or str(uuid.uuid4())
            np.save(enc_path_for(sid), encs[0])
            FACE_DB[sid] = {
                "name": name,
                "uuid": uid,
                "encoding_path": enc_path_for(sid)
            }
            save_index()
            messagebox.showinfo("Success", f"Face saved for {name}.")
            cam._on_close()

        ttk.Button(cam.buttons, text="Save Face", command=save_face).grid(row=0, column=0, padx=6)
        ttk.Button(cam.buttons, text="Cancel", command=cam._on_close).grid(row=0, column=1, padx=6)


# =========================
# GENERATE QR
# =========================
class GenerateQRDialog(tk.Toplevel):
    def _init_(self, master):
        super()._init_(master)
        self.title("Generate QR")
        self.resizable(False, False)
        self.protocol("WM_DELETE_WINDOW", self.destroy)

        frm = ttk.Frame(self, padding=12)
        frm.pack(fill="both", expand=True)

        ttk.Label(frm, text="Student ID:").grid(row=0, column=0, sticky="e", padx=(0, 8), pady=6)
        self.sid_var = tk.StringVar()
        ttk.Entry(frm, textvariable=self.sid_var, width=28).grid(row=0, column=1, pady=6)

        btns = ttk.Frame(frm)
        btns.grid(row=1, column=0, columnspan=2, pady=(8, 0))
        ttk.Button(btns, text="Generate", command=self._generate).grid(row=0, column=0, padx=6)
        ttk.Button(btns, text="Close", command=self.destroy).grid(row=0, column=1, padx=6)

    def _generate(self):
        sid = self.sid_var.get().strip()
        if not sid:
            messagebox.showerror("Error", "Student ID is required.")
            return
        if sid not in FACE_DB or not os.path.exists(FACE_DB[sid].get("encoding_path", "")):
            messagebox.showerror("Error", "No face data found for this Student ID.")
            return

        uid = FACE_DB[sid].get("uuid") or str(uuid.uuid4())
        FACE_DB[sid]["uuid"] = uid

        img = qrcode.make(uid)
        qr_path = os.path.join(QR_DIR, f"{sid}.png")
        img.save(qr_path)
        FACE_DB[sid]["qr"] = qr_path
        save_index()

        messagebox.showinfo("Success", f"QR generated and saved to:\n{qr_path}")


# =========================
# ATTENDANCE (QR → FACE)
# =========================
class AttendanceWindow(CameraView):
    def _init_(self, master):
        super()._init_(master, title="Attendance (QR → Face)")
        self.detector = cv2.QRCodeDetector()
        self.stage = "qr"
        self.matched_sid = None
        self.matched_info = None
        self.stored_encoding = None
        self.set_status("Show your QR code to the camera.")

        ttk.Button(self.buttons, text="Cancel", command=self._on_close).grid(row=0, column=0, padx=6)

        self.set_callback(self._process_frame)

    def _process_frame(self, frame_bgr):
        if self.stage == "qr":
            data, bbox, _ = self.detector.detectAndDecode(frame_bgr)
            if data:
                sid, info = load_by_uuid(data.strip())
                if sid is None:
                    self.set_status("QR not found in database. Try again.")
                else:
                    self.matched_sid = sid
                    self.matched_info = info
                    encp = info.get("encoding_path")
                    if not encp or not os.path.exists(encp):
                        self.set_status("Encoding missing for this student. Re-register face.")
                    else:
                        self.stored_encoding = np.load(encp)
                        self.stage = "face"
                        self.set_status(f"QR OK: {info['name']} ({sid}). Now look at the camera.")
            return frame_bgr

        elif self.stage == "face":
            rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
            locs = face_recognition.face_locations(rgb)
            for (t, r, b, l) in locs:
                cv2.rectangle(frame_bgr, (l, t), (r, b), (0, 255, 0), 2)

            if len(locs) == 0:
                self.set_status("No face detected. Keep only ONE face in view.")
            elif len(locs) > 1:
                self.set_status("Multiple faces detected! Only one person should be in view.")
            else:
                encs = face_recognition.face_encodings(rgb, locs)
                if encs:
                    is_match = face_recognition.compare_faces(
                        [self.stored_encoding], encs[0], tolerance=0.45
                    )[0]
                    if is_match:
                        name = self.matched_info["name"]
                        sid = self.matched_sid
                        cv2.putText(frame_bgr, f"{name} ({sid}) - Present",
                                    (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        self.set_status("Attendance recorded. Closing in 2 seconds...")
                        self.after(2000, self._on_close)
                    else:
                        cv2.putText(frame_bgr, "Face MISMATCH", (20, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                        self.set_status("Face does NOT match QR. Try again.")
            return frame_bgr

        return frame_bgr


# =========================
# COMPARE FACE (LIVE → DB)
# =========================
class CompareFaceWindow(CameraView):
    def _init_(self, master):
        super()._init_(master, title="Compare Face with Database")
        self.set_status("Look at the camera to compare your face.")

        ttk.Button(self.buttons, text="Cancel", command=self._on_close).grid(row=0, column=0, padx=6)

        # Load encodings from DB
        self.encodings = []
        self.metadata = []
        for sid, info in FACE_DB.items():
            encp = info.get("encoding_path")
            if encp and os.path.exists(encp):
                self.encodings.append(np.load(encp))
                self.metadata.append((sid, info["name"]))

        self.set_callback(self._process_frame)

    def _process_frame(self, frame_bgr):
        rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        locs = face_recognition.face_locations(rgb)

        for (t, r, b, l) in locs:
            cv2.rectangle(frame_bgr, (l, t), (r, b), (0, 255, 0), 2)

        if len(locs) == 1:
            encs = face_recognition.face_encodings(rgb, locs)
            if encs:
                matches = face_recognition.compare_faces(self.encodings, encs[0], tolerance=0.45)
                if True in matches:
                    idx = matches.index(True)
                    sid, name = self.metadata[idx]
                    cv2.putText(frame_bgr, f"{name} ({sid})", (20, 40),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    self.set_status(f"Matched: {name} ({sid})")
                else:
                    cv2.putText(frame_bgr, "Unknown", (20, 40),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    self.set_status("No match found.")
        elif len(locs) > 1:
            self.set_status("Multiple faces detected! Only one allowed.")
        else:
            self.set_status("No face detected.")

        return frame_bgr

# =========================
# MAIN APP
# =========================
class App(tk.Tk):
    def _init_(self):
        super()._init_()
        self.title("Face Recognition Attendance System")
        self.geometry("420x300")
        self.resizable(False, False)

        ttk.Style().configure("TButton", padding=6, font=("Arial", 12))
        ttk.Style().configure("H.TLabel", font=("Arial", 16, "bold"))

        header = ttk.Label(self, text="Select an Option", style="H.TLabel")
        header.pack(pady=(20, 10))

        btn_box = ttk.Frame(self)
        btn_box.pack(pady=10)

        ttk.Button(btn_box, text="1) Save Face Data", width=28, command=self.open_register).pack(pady=6)
        ttk.Button(btn_box, text="2) Generate QR Code", width=28, command=self.open_qr).pack(pady=6)
        ttk.Button(btn_box, text="3) Take Attendance (QR → Face)", width=28, command=self.open_attendance).pack(pady=6)
        ttk.Button(btn_box, text="4) Compare Face (Live → DB)", width=28, command=self.open_compare).pack(pady=6)

        ttk.Button(btn_box, text="Exit", width=28, command=self.quit).pack(pady=6)

        self.msg = tk.StringVar(value="")
        ttk.Label(self, textvariable=self.msg).pack(pady=(6, 0))

    def open_register(self):
        RegisterFaceDialog(self)

    def open_qr(self):
        GenerateQRDialog(self)

    def open_attendance(self):
        AttendanceWindow(self)

    def open_compare(self):
        CompareFaceWindow(self)



if _name_ == "_main_":
    load_index()
    App().mainloop()

# Template Matching for face

In [ ]:
##template matching 
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import os
import uuid
import cv2
import numpy as np
from PIL import Image, ImageTk
import qrcode
import json

# =========================
# CONFIG
# =========================
FACE_DB = {}  # sid -> { name, uuid, template_path, qr }
DATA_FILE = "face_db.json"
TEMPLATE_DIR = "templates"
QR_DIR = "qr_codes"

os.makedirs(TEMPLATE_DIR, exist_ok=True)
os.makedirs(QR_DIR, exist_ok=True)

# =========================
# DATA PERSISTENCE
# =========================
def save_index():
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(FACE_DB, f, indent=2)

def load_index():
    global FACE_DB
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, "r", encoding="utf-8") as f:
            FACE_DB = json.load(f)
    else:
        FACE_DB = {}

def template_path_for(sid):
    return os.path.join(TEMPLATE_DIR, f"{sid}.png")

def ensure_single_face(locs):
    return len(locs) == 1

def load_by_uuid(uid):
    for sid, info in FACE_DB.items():
        if info.get("uuid") == uid:
            return sid, info
    return None, None

# =========================
# CAMERA VIEW BASE
# =========================
class CameraView(tk.Toplevel):
    def _init_(self, master, title="Camera"):
        super()._init_(master)
        self.title(title)
        self.protocol("WM_DELETE_WINDOW", self._on_close)
        self.frame = None
        self.callback = None

        self.video_label = ttk.Label(self)
        self.video_label.pack()

        self.status_var = tk.StringVar()
        self.status_label = ttk.Label(self, textvariable=self.status_var)
        self.status_label.pack()

        self.buttons = ttk.Frame(self)
        self.buttons.pack(pady=6)

        self.cap = cv2.VideoCapture(0)
        self._update_frame()

    def set_status(self, text):
        self.status_var.set(text)

    def set_callback(self, func):
        self.callback = func

    def _update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            self.frame = frame.copy()
            display_frame = frame
            if self.callback:
                display_frame = self.callback(frame.copy())
            rgb = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
        self.after(30, self._update_frame)

    def _on_close(self):
        if hasattr(self, "cap") and self.cap.isOpened():
            self.cap.release()
        self.destroy()

# =========================
# FACE DETECTION (HAAR)
# =========================
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

def detect_faces(gray):
    return face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80)
    )

def preprocess_face(frame_bgr, rect):
    (x, y, w, h) = rect
    face = frame_bgr[y:y+h, x:x+w]
    gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (100, 100))
    return resized

# =========================
# REGISTER FACE
# =========================
class RegisterFaceDialog(tk.Toplevel):
    def _init_(self, master):
        super()._init_(master)
        self.title("Register Face")
        self.resizable(False, False)
        self.protocol("WM_DELETE_WINDOW", self.destroy)

        frm = ttk.Frame(self, padding=12)
        frm.pack(fill="both", expand=True)

        ttk.Label(frm, text="Student ID:").grid(row=0, column=0, sticky="e", padx=(0, 8), pady=6)
        ttk.Label(frm, text="Name:").grid(row=1, column=0, sticky="e", padx=(0, 8), pady=6)

        self.sid_var = tk.StringVar()
        self.name_var = tk.StringVar()
        ttk.Entry(frm, textvariable=self.sid_var, width=28).grid(row=0, column=1, pady=6)
        ttk.Entry(frm, textvariable=self.name_var, width=28).grid(row=1, column=1, pady=6)

        self.photo_label = ttk.Label(frm)
        self.photo_label.grid(row=3, column=0, columnspan=2, pady=(10, 0))

        btns = ttk.Frame(frm)
        btns.grid(row=2, column=0, columnspan=2, pady=(10, 0))
        ttk.Button(btns, text="Use Photo", command=self._use_photo).grid(row=0, column=0, padx=6)
        ttk.Button(btns, text="Scan with Camera", command=self._scan_camera).grid(row=0, column=1, padx=6)
        ttk.Button(btns, text="Close", command=self.destroy).grid(row=0, column=2, padx=6)

    def _validate_inputs(self):
        sid = self.sid_var.get().strip()
        name = self.name_var.get().strip()
        if not sid or not name:
            messagebox.showerror("Error", "Student ID and Name are required.")
            return None, None
        return sid, name

    def _use_photo(self):
        sid, name = self._validate_inputs()
        if not sid:
            return

        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
        if not file_path:
            return

        img = cv2.imread(file_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detect_faces(gray)
        if not ensure_single_face(faces):
            messagebox.showerror("Error", "Need exactly one face in the photo.")
            return

        face_img = preprocess_face(img, faces[0])
        cv2.imwrite(template_path_for(sid), face_img)

        uid = FACE_DB.get(sid, {}).get("uuid") or str(uuid.uuid4())
        FACE_DB[sid] = {
            "name": name,
            "uuid": uid,
            "template_path": template_path_for(sid),
            **({"qr": FACE_DB[sid].get("qr")} if sid in FACE_DB else {}),
        }
        save_index()

        thumb = Image.fromarray(face_img).resize((150, 150))
        photo = ImageTk.PhotoImage(thumb)
        self.photo_label.config(image=photo)
        self.photo_label.image = photo
        messagebox.showinfo("Success", "Photo uploaded and face data saved!")

    def _scan_camera(self):
        sid, name = self._validate_inputs()
        if not sid:
            return

        cam = CameraView(self, title="Register Face - Camera")
        cam.set_status("Ensure only ONE face is visible. Click 'Save Face' when ready.")

        def on_frame(frame_bgr):
            gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
            faces = detect_faces(gray)
            for (x, y, w, h) in faces:
                cv2.rectangle(frame_bgr, (x, y), (x+w, y+h), (0, 255, 0), 2)
            return frame_bgr

        cam.set_callback(on_frame)

        def save_face():
            if cam.frame is None:
                return
            gray = cv2.cvtColor(cam.frame, cv2.COLOR_BGR2GRAY)
            faces = detect_faces(gray)
            if not ensure_single_face(faces):
                messagebox.showerror("Error", "Need exactly one face to save.")
                return
            face_img = preprocess_face(cam.frame, faces[0])
            cv2.imwrite(template_path_for(sid), face_img)

            uid = FACE_DB.get(sid, {}).get("uuid") or str(uuid.uuid4())
            FACE_DB[sid] = {"name": name, "uuid": uid, "template_path": template_path_for(sid)}
            save_index()
            messagebox.showinfo("Success", f"Face saved for {name}.")
            cam._on_close()

        ttk.Button(cam.buttons, text="Save Face", command=save_face).grid(row=0, column=0, padx=6)
        ttk.Button(cam.buttons, text="Cancel", command=cam._on_close).grid(row=0, column=1, padx=6)

# =========================
# GENERATE QR
# =========================
class GenerateQRDialog(tk.Toplevel):
    def _init_(self, master):
        super()._init_(master)
        self.title("Generate QR")
        self.resizable(False, False)
        self.protocol("WM_DELETE_WINDOW", self.destroy)

        frm = ttk.Frame(self, padding=12)
        frm.pack(fill="both", expand=True)

        ttk.Label(frm, text="Student ID:").grid(row=0, column=0, sticky="e", padx=(0, 8), pady=6)
        self.sid_var = tk.StringVar()
        ttk.Entry(frm, textvariable=self.sid_var, width=28).grid(row=0, column=1, pady=6)

        btns = ttk.Frame(frm)
        btns.grid(row=1, column=0, columnspan=2, pady=(8, 0))
        ttk.Button(btns, text="Generate", command=self._generate).grid(row=0, column=0, padx=6)
        ttk.Button(btns, text="Close", command=self.destroy).grid(row=0, column=1, padx=6)

    def _generate(self):
        sid = self.sid_var.get().strip()
        if not sid:
            messagebox.showerror("Error", "Student ID is required.")
            return
        if sid not in FACE_DB or not os.path.exists(FACE_DB[sid].get("template_path", "")):
            messagebox.showerror("Error", "No face data found for this Student ID.")
            return

        uid = FACE_DB[sid].get("uuid") or str(uuid.uuid4())
        FACE_DB[sid]["uuid"] = uid
        img = qrcode.make(uid)
        qr_path = os.path.join(QR_DIR, f"{sid}.png")
        img.save(qr_path)
        FACE_DB[sid]["qr"] = qr_path
        save_index()
        messagebox.showinfo("Success", f"QR generated and saved to:\n{qr_path}")

# =========================
# ATTENDANCE (QR → FACE)
# =========================
class AttendanceWindow(CameraView):
    def _init_(self, master):
        super()._init_(master, title="Attendance (QR → Face)")
        self.detector = cv2.QRCodeDetector()
        self.stage = "qr"
        self.matched_sid = None
        self.matched_info = None
        self.stored_template = None
        self.set_status("Show your QR code to the camera.")
        ttk.Button(self.buttons, text="Cancel", command=self._on_close).grid(row=0, column=0, padx=6)
        self.set_callback(self._process_frame)

    def _process_frame(self, frame_bgr):
        if self.stage == "qr":
            data, bbox, _ = self.detector.detectAndDecode(frame_bgr)
            if data:
                sid, info = load_by_uuid(data.strip())
                if sid is None:
                    self.set_status("QR not found in database. Try again.")
                else:
                    self.matched_sid = sid
                    self.matched_info = info
                    tpath = info.get("template_path")
                    if not tpath or not os.path.exists(tpath):
                        self.set_status("Template missing. Re-register face.")
                    else:
                        self.stored_template = cv2.imread(tpath, cv2.IMREAD_GRAYSCALE)
                        self.stage = "face"
                        self.set_status(f"QR OK: {info['name']} ({sid}). Now look at the camera.")
            return frame_bgr

        elif self.stage == "face":
            gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
            faces = detect_faces(gray)
            for (x, y, w, h) in faces:
                cv2.rectangle(frame_bgr, (x, y), (x+w, y+h), (0, 255, 0), 2)

            if len(faces) == 0:
                self.set_status("No face detected.")
            elif len(faces) > 1:
                self.set_status("Multiple faces detected!")
            else:
                face_img = preprocess_face(frame_bgr, faces[0])
                res = cv2.matchTemplate(face_img, self.stored_template, cv2.TM_CCOEFF_NORMED)
                score = res[0][0]
                if score > 0.7:
                    name = self.matched_info["name"]
                    sid = self.matched_sid
                    cv2.putText(frame_bgr, f"{name} ({sid}) - Present", (20, 40),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    self.set_status("Attendance recorded. Closing in 2 seconds...")
                    self.after(2000, self._on_close)
                else:
                    cv2.putText(frame_bgr, "Face MISMATCH", (20, 40),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    self.set_status("Face does NOT match QR. Try again.")
            return frame_bgr

        return frame_bgr

# =========================
# COMPARE FACE (LIVE → DB)
# =========================
class CompareFaceWindow(CameraView):
    def _init_(self, master):
        super()._init_(master, title="Compare Face with Database")
        self.set_status("Look at the camera to compare your face.")
        ttk.Button(self.buttons, text="Cancel", command=self._on_close).grid(row=0, column=0, padx=6)

        # Load templates
        self.templates = []
        self.metadata = []
        for sid, info in FACE_DB.items():
            tpath = info.get("template_path")
            if tpath and os.path.exists(tpath):
                self.templates.append(cv2.imread(tpath, cv2.IMREAD_GRAYSCALE))
                self.metadata.append((sid, info["name"]))

        self.set_callback(self._process_frame)

    def _process_frame(self, frame_bgr):
        gray = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2GRAY)
        faces = detect_faces(gray)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame_bgr, (x, y), (x+w, y+h), (0, 255, 0), 2)

        if len(faces) == 1:
            face_img = preprocess_face(frame_bgr, faces[0])
            best_score = 0
            best_match = None
            for idx, tmpl in enumerate(self.templates):
                res = cv2.matchTemplate(face_img, tmpl, cv2.TM_CCOEFF_NORMED)
                score = res[0][0]
                if score > best_score:
                    best_score = score
                    best_match = self.metadata[idx]
            if best_match and best_score > 0.7:
                sid, name = best_match
                cv2.putText(frame_bgr, f"{name} ({sid})", (20, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                self.set_status(f"Matched: {name} ({sid}) [score={best_score:.2f}]")
            else:
                cv2.putText(frame_bgr, "Unknown", (20, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                self.set_status("No match found.")
        elif len(faces) > 1:
            self.set_status("Multiple faces detected! Only one allowed.")
        else:
            self.set_status("No face detected.")
        return frame_bgr

# =========================
# MAIN APP
# =========================
class App(tk.Tk):
    def _init_(self):
        super()._init_()
        self.title("Face Recognition (Template Matching)")
        self.geometry("420x300")
        self.resizable(False, False)

        ttk.Style().configure("TButton", padding=6, font=("Arial", 12))
        ttk.Style().configure("H.TLabel", font=("Arial", 16, "bold"))

        header = ttk.Label(self, text="Select an Option", style="H.TLabel")
        header.pack(pady=(20, 10))

        btn_box = ttk.Frame(self)
        btn_box.pack(pady=10)

        ttk.Button(btn_box, text="1) Save Face Data", width=28, command=self.open_register).pack(pady=6)
        ttk.Button(btn_box, text="2) Generate QR Code", width=28, command=self.open_qr).pack(pady=6)
        ttk.Button(btn_box, text="3) Take Attendance (QR → Face)", width=28, command=self.open_attendance).pack(pady=6)
        ttk.Button(btn_box, text="4) Compare Face (Live → DB)", width=28, command=self.open_compare).pack(pady=6)
        ttk.Button(btn_box, text="Exit", width=28, command=self.quit).pack(pady=6)

        self.msg = tk.StringVar(value="")
        ttk.Label(self, textvariable=self.msg).pack(pady=(6, 0))

    def open_register(self):
        RegisterFaceDialog(self)

    def open_qr(self):
        GenerateQRDialog(self)

    def open_attendance(self):
        AttendanceWindow(self)

    def open_compare(self):
        CompareFaceWindow(self)

if _name_ == "_main_":
    load_index()
    App().mainloop()
